In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import argparse
import os

In [41]:
def get_region_means(regions, df):
  chrset = ["chr1", "chr2", "chr3", "chr4", "chr5", "chr6", "chr7", "chr8", "chr9", "chr10","chr11","chr12","chr13", "chr14", "chr15", "chr16", "chr17","chr18", "chr19", "chr20", "chr21", "chr22", "chrX", "chrY"]
  results_mean = pd.DataFrame()

  #Main Region Function
  first = True
  for chr in chrset:
    chr_regions = pd.DataFrame()
    df_regions = pd.DataFrame()
    chr_regions = regions[regions["CHR"] == chr]
    df_regions = df[df["CHR"] == chr]
    #print(df_regions)
    
    for index in chr_regions.index:
      #create temp array for holding start and stop values of a region
      ## Decide here if you want NEW (trimmed) or OLD regions
      Start = chr_regions.loc[index, "Start"]
      Stop = chr_regions.loc[index, "Stop"]
      #select for only CpG's within the given region
      r = df_regions[(df_regions["Location"] >= Start) & (df_regions["Location"] <= Stop)]
      r = r.drop(["CHR", "Location"], axis = 1)
      ### r = r.drop(["CHR", "Location", "CHR.1", "Location.1"], axis = 1)
      #track the number of CpG's in given region
      l = len((r.index))
      r = r.apply(np.nanmean) #remove na's ??
      #calculate the mean of the row, rewrite itself
      anno_series = pd.Series(dtype=object)
      anno_series["Name"] = f"{chr}:{int(Start)}-{int(Stop)}"
      anno_series["CHR"] = chr
      anno_series["Start"] = Start
      anno_series["Stop"] = Stop
      anno_series["CPG_Count"] = l
      r = pd.concat([anno_series,r])
      r = (r.to_frame()).T 
      if first == True:
        results_mean= r
        first = False
      else:
        results_mean = pd.concat([results_mean,r], axis=0)

  results_mean.index = results_mean["Name"]
  results_mean = results_mean.drop(["Name"],axis=1)
  results_mean = results_mean.rename_axis(None, axis=0) #remove index title

  #Let's remove all the extra info
  clean_results_mean = results_mean.iloc[:,4:]
  return clean_results_mean

In [50]:
betas = pd.read_csv("/Users/carternorton/Desktop/Contamination_Array/Contamination_Array_beta_values.csv",header=0, index_col=0)

#Drop any CpG's with NA values
betas = betas.dropna(axis=0, how='any')

betas.iloc[:]

,9611519066_R01C01,9611519066_R02C01,9611519066_R03C01,9611519066_R04C01,9611519066_R05C01,9611519066_R06C01,9611519066_R01C02,9611519066_R02C02,9611519066_R03C02,9611519066_R04C02,...,9611518122_R03C01,9611518122_R04C01,9611518122_R05C01,9611518122_R06C01,9611518122_R01C02,9611518122_R02C02,9611518122_R03C02,9611518122_R04C02,9611518122_R05C02,9611518122_R06C02
cg00050873,0.083082,0.770541,0.231055,0.441520,0.107962,0.481344,0.089090,0.102937,0.099198,0.305314,...,0.302845,0.419297,0.091247,0.515162,0.091173,0.115780,0.099295,0.274474,0.436300,0.171232
cg00212031,0.033611,0.019297,0.038859,0.029688,0.023760,0.048070,0.033190,0.031376,0.021857,0.033720,...,0.031769,0.028454,0.031476,0.026809,0.026345,0.027764,0.025913,0.025401,0.030295,0.024914
cg00213748,0.045641,0.799692,0.096312,0.224559,0.085836,0.295331,0.055563,0.060929,0.073130,0.102838,...,0.081100,0.221792,0.087612,0.289489,0.056908,0.060253,0.056361,0.085941,0.186341,0.077600
cg00214611,0.030005,0.040821,0.030113,0.055613,0.055351,0.039021,0.029993,0.038520,0.034693,0.027332,...,0.030145,0.028083,0.036068,0.028461,0.027302,0.029382,0.024126,0.023281,0.021973,0.030915
cg00455876,0.077259,0.609653,0.165104,0.318574,0.101371,0.400725,0.062812,0.053249,0.069320,0.185857,...,0.169329,0.339840,0.080170,0.346025,0.063647,0.114372,0.081150,0.159331,0.325418,0.127155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ch.22.909671F,0.086955,0.097113,0.148309,0.124401,0.163855,0.146428,0.142083,0.161414,0.206565,0.105122,...,0.104332,0.081501,0.118235,0.092902,0.128435,0.111218,0.117692,0.093012,0.050893,0.053821
ch.22.46830341F,0.014698,0.038371,0.035262,0.041925,0.039957,0.033040,0.024278,0.024921,0.035469,0.032605,...,0.034511,0.040426,0.043550,0.031123,0.025007,0.030928,0.033667,0.036676,0.044965,0.044183
ch.22.1008279F,0.016523,0.019392,0.022083,0.018589,0.021094,0.026018,0.022827,0.019889,0.026267,0.024447,...,0.020002,0.026347,0.019721,0.024256,0.017656,0.022215,0.010923,0.020528,0.013041,0.022069
ch.22.47579720R,0.067125,0.070926,0.061106,0.067779,0.060168,0.084182,0.061654,0.092231,0.053435,0.096871,...,0.069754,0.077983,0.076858,0.059910,0.074810,0.078795,0.108209,0.081662,0.080359,0.073001


In [51]:
dmrs = pd.read_csv("/Users/carternorton/Desktop/Somatic_Cell_ML_Pipeline/DMR_Files/DMRs.csv",header=0, index_col=0)
dmrs

,CHR,Start,Stop
chr1:167604788-167604789,chr1,167604788,167604789
chr1:248596843-248597494,chr1,248596843,248597494
chr1:244303292-244303506,chr1,244303292,244303506
chr1:167656457-167656458,chr1,167656457,167656458
chr1:202856391-202856392,chr1,202856391,202856392
...,...,...,...
chrX:49236806-49236807,chrX,49236806,49236807
chrX:49085918-49085919,chrX,49085918,49085919
chrX:109936239-109936240,chrX,109936239,109936240
chrX:12974205-12975051,chrX,12974205,12975051


In [52]:
cutoffs = pd.read_csv("/Users/carternorton/Desktop/Somatic_Cell_ML_Pipeline/DMR_Files/cutoffs.csv",header=0,index_col=0)
cutoffs

,mean,std,mean+2std,mean-2std
chr1:167604788-167604789,0.030764,0.025336,0.081436,-0.019907
chr1:59207489-59207490,0.044598,0.016321,0.077239,0.011956
chr1:246487983-246487984,0.034523,0.022912,0.080348,-0.011302
chr1:199704584-199704585,0.040715,0.014708,0.070131,0.011299
chr1:221737032-221737207,0.023492,0.012230,0.047952,-0.000967
...,...,...,...,...
chrX:70416006-70416007,0.912262,0.018853,0.949968,0.874557
chrX:12974205-12975051,0.929274,0.015549,0.960372,0.898176
chrX:18874432-18874433,0.912361,0.017158,0.946677,0.878044
chrY:21208621-21208622,0.093015,0.023563,0.140142,0.045888


In [53]:
if len(betas) > 500000:
    print("***** This looks like EPIC 850K Data ****")
    data_type = "EPIC"
if len(betas) < 500000 and len(betas) > 30000:
    print("**** This looks like 450K Data ****")
    data_type = "450K"
if len(betas) < 30000:
    print("This looks like 27K data. REMINDER THIS PIPELINE WILL NOT WORK FOR ANY OTHER PLATFORM BUT 450K AND 850K METHYLATION ARRAY DATA")


**** This looks like 450K Data ****


In [54]:
#Let's load in the annotation files
if data_type == "EPIC":
    anno = pd.read_csv("/Users/carternorton/Desktop/Somatic_Cell_ML_Pipeline/Anno/epic_anno.csv", index_col = 0)
if data_type == "450K":
    anno = pd.read_csv("/Users/carternorton/Desktop/Somatic_Cell_ML_Pipeline/Anno/450k_Anno.csv", index_col = 0)


In [55]:
#Let's merge the betas and annotation files
anno_betas = pd.merge(anno, betas, left_index=True, right_index=True)
anno_betas

,CHR,Location,9611519066_R01C01,9611519066_R02C01,9611519066_R03C01,9611519066_R04C01,9611519066_R05C01,9611519066_R06C01,9611519066_R01C02,9611519066_R02C02,...,9611518122_R03C01,9611518122_R04C01,9611518122_R05C01,9611518122_R06C01,9611518122_R01C02,9611518122_R02C02,9611518122_R03C02,9611518122_R04C02,9611518122_R05C02,9611518122_R06C02
cg13869341,chr1,15865.0,0.795999,0.751934,0.776677,0.770782,0.811015,0.832033,0.771693,0.778475,...,0.824959,0.824544,0.841528,0.874725,0.806136,0.819191,0.809405,0.824354,0.821314,0.858134
cg14008030,chr1,18827.0,0.757995,0.581857,0.752515,0.668848,0.776882,0.700428,0.790833,0.781890,...,0.694845,0.674303,0.738227,0.681910,0.731458,0.696739,0.733880,0.719118,0.668545,0.741660
cg12045430,chr1,29407.0,0.025788,0.058574,0.056556,0.057953,0.043606,0.045150,0.030991,0.037540,...,0.047811,0.061550,0.040045,0.043080,0.043383,0.053551,0.042255,0.062043,0.067798,0.054850
cg20826792,chr1,29425.0,0.053860,0.159640,0.076718,0.126138,0.055783,0.132390,0.046203,0.045619,...,0.153052,0.201666,0.049429,0.166320,0.051157,0.069376,0.057235,0.147136,0.201022,0.111700
cg00381604,chr1,29435.0,0.040348,0.047375,0.050520,0.050269,0.056404,0.059242,0.046996,0.043363,...,0.050830,0.057403,0.061118,0.051964,0.055524,0.059352,0.075089,0.039685,0.047124,0.065762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cg17939569,chrY,27009430.0,0.927245,0.692268,0.888886,0.833864,0.939961,0.874147,0.934297,0.940100,...,0.898543,0.798642,0.964034,0.853943,0.915837,0.933529,0.956682,0.920145,0.817306,0.933902
cg13365400,chrY,27210334.0,0.830108,0.718722,0.827448,0.799222,0.861876,0.837699,0.815730,0.860186,...,0.847653,0.843145,0.900587,0.863049,0.877233,0.831488,0.870679,0.869568,0.840942,0.894912
cg21106100,chrY,28555536.0,0.021825,0.858831,0.214863,0.486012,0.034894,0.617296,0.016951,0.042180,...,0.284783,0.505115,0.038079,0.689854,0.031049,0.051672,0.030195,0.238119,0.469915,0.194297
cg08265308,chrY,28555550.0,0.049070,0.875000,0.101258,0.339525,0.057948,0.480235,0.042945,0.073074,...,0.185720,0.404485,0.036121,0.583769,0.045125,0.066396,0.049963,0.131342,0.345464,0.139723


In [56]:
#Let's get the mean beta values for each region
mean_betas = get_region_means(dmrs, anno_betas)
mean_betas

,9611519066_R01C01,9611519066_R02C01,9611519066_R03C01,9611519066_R04C01,9611519066_R05C01,9611519066_R06C01,9611519066_R01C02,9611519066_R02C02,9611519066_R03C02,9611519066_R04C02,...,9611518122_R03C01,9611518122_R04C01,9611518122_R05C01,9611518122_R06C01,9611518122_R01C02,9611518122_R02C02,9611518122_R03C02,9611518122_R04C02,9611518122_R05C02,9611518122_R06C02
chr1:167604788-167604789,0.024866,0.89034,0.128993,0.315893,0.033641,0.437398,0.033303,0.036529,0.032172,0.18444,...,0.172457,0.353926,0.024191,0.455494,0.023233,0.038593,0.013827,0.165206,0.353989,0.086197
chr1:248596843-248597494,0.040441,0.928893,0.145184,0.351688,0.045864,0.442149,0.036865,0.048411,0.043547,0.179115,...,0.174807,0.350673,0.033865,0.467331,0.043054,0.046516,0.034131,0.174293,0.373414,0.102719
chr1:244303292-244303506,0.022571,0.918712,0.100344,0.381558,0.040095,0.461246,0.042925,0.042594,0.045779,0.154946,...,0.166861,0.380649,0.025447,0.522105,0.036034,0.040949,0.044931,0.160786,0.383202,0.067779
chr1:167656457-167656458,0.041595,0.934797,0.134155,0.375415,0.030123,0.461322,0.040118,0.037131,0.045287,0.195774,...,0.193998,0.388006,0.032241,0.583684,0.020992,0.035805,0.03023,0.154968,0.393048,0.08867
chr1:202856391-202856392,0.043382,0.917994,0.170228,0.521642,0.067461,0.617908,0.087827,0.105278,0.162522,0.184565,...,0.26319,0.507535,0.119261,0.715465,0.079694,0.065138,0.055485,0.192771,0.513679,0.118124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrX:49236806-49236807,0.036327,0.859261,0.198466,0.379176,0.102067,0.396558,0.160856,0.157693,0.121767,0.242649,...,0.269942,0.392794,0.112658,0.473422,0.079344,0.114202,0.148078,0.215297,0.419387,0.198047
chrX:49085918-49085919,0.048292,0.954699,0.281865,0.546959,0.106908,0.678829,0.07363,0.156805,0.098015,0.244891,...,0.305942,0.547664,0.131175,0.771225,0.134894,0.068933,0.058691,0.245111,0.567924,0.186886
chrX:109936239-109936240,0.954219,0.028225,0.916713,0.77225,0.939186,0.753705,0.919427,0.95986,0.938925,0.886696,...,0.908711,0.741791,0.959966,0.685707,0.968626,0.953839,0.950464,0.910042,0.751035,0.921526
chrX:12974205-12975051,0.928759,0.028335,0.846354,0.585298,0.93434,0.543011,0.925644,0.914296,0.931064,0.815652,...,0.825356,0.630192,0.944521,0.511392,0.937298,0.928032,0.924256,0.866414,0.616109,0.907795


In [63]:
#Create binary dataframes for all samples in each dataframe
def getBinary(df, summary):
    output = pd.DataFrame(index=df.index, columns=df.columns)

    #drop all summary columns that are not in df
    summary = summary[summary.index.isin(df.index)]

    for i, row in df.iterrows():
        for j, x in row.iteritems():
            if (x > summary.loc[i, "mean+2std"]) or (x < summary.loc[i, "mean-2std"]):
                output.loc[i, j] = 1
            else:
                output.loc[i, j] = 0
    return output

beta_binary = getBinary(mean_betas, cutoffs).T
beta_binary

,chr1:167604788-167604789,chr1:248596843-248597494,chr1:244303292-244303506,chr1:167656457-167656458,chr1:202856391-202856392,chr1:246310349-246310486,chr1:25919290-25919395,chr1:244199712-244199782,chr1:227753056-227753057,chr1:210032146-210032147,...,chr22:25056417-25056418,chr22:38001083-38001084,chr22:40420987-40420988,chrX:84581306-84581307,chrX:96138767-96139068,chrX:49236806-49236807,chrX:49085918-49085919,chrX:109936239-109936240,chrX:12974205-12975051,chrY:21208621-21208622
9611519066_R01C01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9611519066_R02C01,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9611519066_R03C01,1,1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,0,1,1
9611519066_R04C01,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9611519066_R05C01,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
9611519066_R06C01,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9611519066_R01C02,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
9611519066_R02C02,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
9611519066_R03C02,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9611519066_R04C02,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1


In [70]:
import pickle

# Open the file containing the pickle object
with open('/Users/carternorton/Desktop/Somatic_Cell_ML_Pipeline/LR_model.sav', 'rb') as file:
    # Load the object from the file
    LR_model = pickle.load(file)

# Use the loaded model to make predictions
threshold = .99999 #Model must be 99.999% sure that the sample is uncontaminated to classify as uncontaminated
proba = LR_model.predict_proba(beta_binary)
binary_pred = ["Uncontaminated" if x[1] > threshold else "Contaminated" for x in proba]

binary_pred = pd.DataFrame(binary_pred, index = beta_binary.index, columns = ["Prediction"])

#Add a probability column to the dataframe
binary_pred["Predicted_Probability_of_Contamination_%"] = [((1 - x[1]) * 100) for x in proba]
binary_pred







,Prediction,Predicted_Probability_of_Contamination_%
9611519066_R01C01,Uncontaminated,0.000034
9611519066_R02C01,Contaminated,99.999240
9611519066_R03C01,Contaminated,30.070053
9611519066_R04C01,Contaminated,99.999240
9611519066_R05C01,Uncontaminated,0.000035
9611519066_R06C01,Contaminated,99.999240
9611519066_R01C02,Uncontaminated,0.000049
9611519066_R02C02,Uncontaminated,0.000093
9611519066_R03C02,Uncontaminated,0.000070
9611519066_R04C02,Contaminated,99.715239
